# Live Model Server Testing

Test model server via HTTP calls 

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config spec.image = "mlrun/mlrun"
%nuclio config spec.maxReplicas = 1

%nuclio: setting spec.image to 'mlrun/mlrun'
%nuclio: setting spec.maxReplicas to 1


In [3]:
import os
import pandas as pd
import requests
import json
import numpy as np
from datetime import datetime
from mlrun.datastore import DataItem
from mlrun.artifacts import ChartArtifact

def model_server_tester(context,
                        table: DataItem,
                        addr: str, 
                        label_column: str = "label",
                        model: str = '',
                        match_err: bool = False,
                        rows: int = 20):
    """ Test a model server 
    
    :param table:         csv/parquet table with test data
    :param addr:          function address/url
    :param label_column:  name of the label column in table
    :param model:         tested model name 
    :param match_err:     raise error on validation (require proper test set)
    :param rows:          number of rows to use from test set
    """
        
    table = table.as_df()

    y_list = table.pop(label_column).values.tolist()
    context.logger.info(f'testing with dataset against {addr}, model: {model}')
    if rows and rows < table.shape[0]:
        table = table.sample(rows)
    
    count = err_count = match = 0
    times = []
    for x, y in zip(table.values, y_list):
        count += 1
        event_data = json.dumps({"inputs":[x.tolist()]})
        had_err = False
        try:
            start = datetime.now()
            resp = requests.put(f'{addr}/v2/models/{model}/infer', json=event_data)
            if not resp.ok:
                context.logger.error(f'bad function resp!!\n{resp.text}')
                err_count += 1
                continue
            times.append((datetime.now()-start).microseconds)
                
        except OSError as err:
            context.logger.error(f'error in request, data:{event_data}, error: {err}')
            err_count += 1
            continue
        
        resp_data = resp.json()
        print(resp_data)
        y_resp = resp_data['outputs'][0]
        if y == y_resp:
            match += 1
        
    context.log_result('total_tests', count)
    context.log_result('errors', err_count)
    context.log_result('match', match)
    if count - err_count > 0:
        times_arr = np.array(times)
        context.log_result('avg_latency', int(np.mean(times_arr)))
        context.log_result('min_latency', int(np.amin(times_arr)))
        context.log_result('max_latency', int(np.amax(times_arr)))
        
        chart = ChartArtifact('latency', header=['Test', 'Latency (microsec)'])
        for i in range(len(times)):
            chart.add_row([i+1, int(times[i])])
        context.log_artifact(chart)

    context.logger.info(f'run {count} tests, {err_count} errors and {match} match expected value')
    
    if err_count:
        raise ValueError(f'failed on {err_count} tests of {count}')
    
    if match_err and match != count:
        raise ValueError(f'only {match} results match out of {count}')

> 2020-10-28 17:05:57,889 [warning] Failed resolving version info. Ignoring and using defaults


In [4]:
# nuclio: end-code
# marks the end of a code section

### Deploy model server for testing

In [5]:
import mlrun

project_name = 'sk-project'
DATA_PATH = 'https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv'
MODEL_PATH = 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'

artifact_path = mlrun.set_environment(api_path = 'http://mlrun-api:8080',
                                      artifact_path = os.path.abspath('./'))

fn = mlrun.import_function('hub://v2_model_server')
fn.add_model('mymodel', model_path=MODEL_PATH)
address = fn.deploy()

> 2020-10-28 17:05:57,914 [warning] warning!, server (0.5.3-rc1) and client (unstable) ver dont match
> 2020-10-28 17:05:58,324 [info] deploy started
[nuclio] 2020-10-28 17:06:08,637 (info) Build complete
[nuclio] 2020-10-28 17:06:11,672 done updating default-v2-model-server, function address: default-tenant.app.dsteam.iguazio-cd1.com:30984
> 2020-10-28 17:06:11,679 [warning] warning!, server (0.5.3-rc1) and client (unstable) ver dont match


### Run model server tester locally

In [6]:
gen = mlrun.run_local(name='model_server_tester', handler=model_server_tester, 
                      params={'addr': address, 'model': 'mymodel'},
                      inputs={'table': DATA_PATH},
                      project=project_name, 
                      artifact_path=os.path.join(artifact_path, 'data')) 

> 2020-10-28 17:06:11,735 [warning] warning!, server (0.5.3-rc1) and client (unstable) ver dont match
> 2020-10-28 17:06:11,736 [info] starting run model_server_tester uid=cdab1ab0ee78491aa7112199edd13eee  -> http://mlrun-api:8080
> 2020-10-28 17:06:11,774 [warning] warning!, server (0.5.3-rc1) and client (unstable) ver dont match
> 2020-10-28 17:06:11,926 [info] testing with dataset against http://default-tenant.app.dsteam.iguazio-cd1.com:30984, model: mymodel
{'id': 'ff2b8b28-a577-41f7-9903-10d30b5e5ce1', 'model_name': 'mymodel', 'outputs': [1]}
{'id': 'de0ebcde-584b-47a1-a3a2-58b71d5dfd6e', 'model_name': 'mymodel', 'outputs': [2]}
{'id': 'b5b48278-1923-4b22-afa0-125d168cf8f2', 'model_name': 'mymodel', 'outputs': [0]}
{'id': 'adc5fa66-d375-4284-956d-258cfccf7ffd', 'model_name': 'mymodel', 'outputs': [0]}
{'id': '34d7aec0-8c00-4a46-beff-3d3f230f1eb6', 'model_name': 'mymodel', 'outputs': [1]}
{'id': '3b54ba8d-2dd9-4f1b-b047-d7dd0795d700', 'model_name': 'mymodel', 'outputs': [2]}
{'id':

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...edd13eee,0,Oct 28 17:06:11,completed,model_server_tester,v3io_user=adminkind=handlerowner=adminhost=jupyter-d87678b84-n4lcf,table,addr=http://default-tenant.app.dsteam.iguazio-cd1.com:30984model=mymodel,total_tests=20errors=0match=6avg_latency=32244min_latency=24955max_latency=116585,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run cdab1ab0ee78491aa7112199edd13eee --project sk-project , !mlrun logs cdab1ab0ee78491aa7112199edd13eee --project sk-project
> 2020-10-28 17:06:12,724 [info] run executed, status=completed


### Save

In [7]:
test_func = mlrun.code_to_function(name='v2_model_tester', 
                                   kind='job', 
                                   handler="model_server_tester",
                                   description="test v2 model servers",
                                   categories=["ml", "test"],
                                   labels={"author": "yaronh"},
                                   code_output='.')

test_func.export('function.yaml')

> 2020-10-28 17:06:21,163 [info] function spec saved to path: function.yaml


### Run remotely

In [8]:
test_func.run(mlrun.NewTask(name='model_server_tester', 
                            handler=model_server_tester, 
                            params={'addr': address, 'model': 'mymodel'},
                            inputs={'table': DATA_PATH},
                            project=project_name, 
                            artifact_path=os.path.join(artifact_path, 'data')))

> 2020-10-28 17:06:21,179 [warning] warning!, server (0.5.3-rc1) and client (unstable) ver dont match
> 2020-10-28 17:06:21,180 [info] starting run model_server_tester uid=daaed7fcf79242e2b9e0c3a5148c64d1  -> http://mlrun-api:8080
> 2020-10-28 17:06:21,307 [info] Job is running in the background, pod: model-server-tester-k2s98
> 2020-10-28 17:06:24,160 [info] testing with dataset against http://default-tenant.app.dsteam.iguazio-cd1.com:30984, model: mymodel
{'id': 'b918f1e7-e376-490a-91a9-d603a8d05c48', 'model_name': 'mymodel', 'outputs': [1]}
{'id': 'f73b94f2-0191-497b-9a0d-b6e31f53753d', 'model_name': 'mymodel', 'outputs': [2]}
{'id': '113a1873-f82e-4c1e-8eb0-e8f93035c841', 'model_name': 'mymodel', 'outputs': [0]}
{'id': 'bf9ccb23-0d77-4f12-b00b-13ed6770c062', 'model_name': 'mymodel', 'outputs': [0]}
{'id': 'acab9cdd-5c10-415d-9658-002b38c8a350', 'model_name': 'mymodel', 'outputs': [2]}
{'id': 'a8f39660-e9c5-4d64-bdde-d0427fbadf0d', 'model_name': 'mymodel', 'outputs': [0]}
{'id': '6d

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...148c64d1,0,Oct 28 17:06:24,completed,model_server_tester,v3io_user=adminkind=jobowner=adminhost=model-server-tester-k2s98,table,addr=http://default-tenant.app.dsteam.iguazio-cd1.com:30984model=mymodel,total_tests=20errors=0match=5avg_latency=26616min_latency=25060max_latency=31181,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run daaed7fcf79242e2b9e0c3a5148c64d1 --project sk-project , !mlrun logs daaed7fcf79242e2b9e0c3a5148c64d1 --project sk-project
> 2020-10-28 17:06:27,426 [info] run executed, status=completed
